In [158]:
# coding: utf-8

# In[1]:


import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from collections import OrderedDict
from scipy.optimize import minimize
from numpy.random import multivariate_normal
from scipy.special import erf
from scipy.integrate import quad
import matplotlib.pyplot as plt
from numpy.linalg import inv
from numpy.random import uniform

from pathlib import Path
Path.ls = lambda p: [i for i in p.iterdir()]


# In[2]:


from scipy.stats import beta

def train_test_split(x, ratio):
    n_train = round(len(x)*ratio)
    x_train = x[0:n_train]
    x_test = x[n_train:]
    return (x_train, x_test)

def integrand(x):
    return np.exp(-x**2)

def ERF(params):
    alpha, beta, p, t = params
    d = (p/2) * (1 + erf(alpha*(t-beta)))
    return d

def ERF2(params):
    alpha, beta, P, t = params
    d = (P/2) * (1 + (2/np.sqrt(np.pi))* quad(integrand, 0, alpha*(t-beta))[0])
    return d


def MSE(params, *args):
    alpha, beta, P = params[0], params[1], params[2]
    ts, Ds, N, loss = args # Ds: True death
    alphas = np.ones(len(ts))*alpha
    betas = np.ones(len(ts))*beta
    Ps = np.ones(len(ts))*P
    params = np.stack((alphas, betas, Ps, ts), axis=-1)
    ds = np.apply_along_axis(ERF, 1, params)
#     ratio1 =  (Ds[-1] - Ds[-2])/(ds[-1] - ds[-2])
#     ratio2 = (ds[-1] - ds[-2])/(Ds[-1] - Ds[-2])
#     factor = (np.log(Ds[-1])-np.log(ds[-1]))**2
    if(loss == 'weight'):
        weights = np.ones(len(Ds))
        weights[-3:]=100
        mse = np.mean((np.log(Ds+1)-np.log(ds+1))**2*Ds**2)
    else:
        mse = np.mean((np.log(Ds+1)-np.log(ds+1))**2)
    return mse


def minimization(params, args):
    # params is alphas, betas, Ps, t
    
    result = minimize(MSE, params, args=args, method = 'L-BFGS-B', bounds = ((0,1), (1, 1e3), (1e0, 1e10)))
    collect = result.x
    success = result.success
    alpha, beta, p = collect[0], collect[1], collect[2]
    ts, Ds, N, loss = args
    alphas = np.ones(len(ts))*alpha
    betas = np.ones(len(ts))*beta
    Ps = np.ones(len(ts))*p
    params = np.stack((alphas, betas, Ps, ts), axis=-1)
    ds = np.apply_along_axis(ERF, 1, params)
    return (ds, alpha, beta, p, success)



def Jacobian(params):
    alpha, beta, p, t = params
    dfda = (p/np.sqrt(np.pi))*np.exp(-alpha**2*(t-beta)**2)*(t-beta)

    dfdb = -(p/np.sqrt(np.pi))*alpha*np.exp(-alpha**2*(t-beta)**2)
    dfdp = ERF(params)/p
    J = np.array([dfda, dfdb, dfdp, 0]).reshape(-1,1)
    return J

# loghessian calculate the hessian of loglik at each time point t
def loghessian(params):
    # diff is the sum of difference between Ds and ds
    alpha, beta, p, t, diff = params
    
    d = ERF((alpha, beta, p, t))
    dfda, dfdb, dfdp, _ = Jacobian((alpha, beta, p, t)).flatten()


    
    J = np.array([dfda, dfdb, dfdp]).reshape(-1,1)
    matrix = J@J.T/d**2
    
    return matrix



def uncertainty(params):
    # t1 is the interpolating series, t2 is the extrapolating series
    alpha, beta, p, t1, t2, sigma2, diff = params
    
    T = len(t1)
    alpha1 = np.ones(len(t1))*alpha
    beta1 = np.ones(len(t1))*beta
    p1 = np.ones(len(t1))*p
    diff1 = diff[0:len(t1)]

    params1 = np.stack((alpha1, beta1, p1, t1, diff1), axis=-1)
    
    
    hess1 = np.sum(np.apply_along_axis(loghessian, 1, params1), axis = 0)/sigma2
    hessian = hess1
    
    
    alpha2 = np.ones(len(t2))*alpha
    beta2 = np.ones(len(t2))*beta
    p2 = np.ones(len(t2))*p
    
    params2 = np.stack((alpha2, beta2, p2, t2), axis=-1)

    d2 = np.apply_along_axis(ERF, 1, params2)

    covariance = inv(hessian)
       
    return covariance


def bootstrap(params):
    D, ts, alpha, beta, p, loss = params
    packs = []
    
    print(alpha, beta, p)
    for i in range(200):
#         Diff = np.concatenate(([0],np.log(D[1:])-np.log(D[:-1])))
#         Diff = beta.pdf((ts+1)/(ts[-1]*1.1), 0.5, 0.5)
#         Diff = np.concatenate(([1], (D[1:] - D[:-1])/D[:-1]))
#         Diff = np.log(D)
        propor = (D[:-1]/np.sum(D[:-1]))
        if loss == 'weight':
            prop = D[len(D)-6:-1]/np.sum(D[len(D)-6:-1])
            t_sample = np.sort(np.random.choice(np.arange(len(D)-6,len(D)-1), 3,p=prop,replace=False))
            t_sample = np.concatenate([t_sample, [len(D)-1]])
        else:
            prop = D[len(D)-6:-1]/np.sum(D[len(D)-6:-1])
            t_sample = np.sort(np.random.choice(np.arange(len(D)-6,len(D)-1), 3,p=prop,replace=False))
            t_sample = np.concatenate([t_sample, [len(D)-1]])
#             t_sample = np.sort(np.random.choice(len(D), int((len(D))*0.6), replace=False))
#             t_sample = np.concatenate([t_sample, [len(D)-1]])
        d_sample = D[t_sample]
        a0 = uniform(alpha*0.1, min(alpha*2,0.1))
        b0 = uniform(0.8*beta, beta*1.2)
        p0 = uniform(0.8*p, 1.2*p)
        _, alpha_hat, beta_hat, p_hat, success = minimization((a0, b0, p0), (t_sample, d_sample, len(d_sample), loss))
        packs.append([alpha_hat, beta_hat, p_hat])
        
    packs = np.array(packs)
    return packs


# In[3]:


def upper_lower(packs, D_sim, ts, CI = 0.95, debug=False):
    Ds = []
    likelis = []
    invalid = 0
    for alpha,beta,p in packs:

        if(alpha <0 or alpha >= 1 or beta <0 or p <0):
            invalid += 1
            print(invalid)
            continue
        if(debug): print(alpha, beta, p)
        D = ERF((alpha, beta, p, ts))


        likelis.append(np.mean((np.log(D_sim+1) - np.log(D+1))**2))

        Ds.append(D)
    Sample = len(Ds)
    print(Sample)

    order = sorted(range(len(likelis)), key=lambda k: likelis[k])
    
    Ds = [Ds[i] for i in order][0:round(Sample*CI)]
    Ds_mat = np.array(Ds)
    D_up =  np.max(Ds_mat,axis=0)
    D_but =  np.min(Ds_mat,axis=0)
    
#     if D_but[0] <= 1:
#         print(D_but)
#         print(packs[Ds_mat[:, 0] <= 1])
#         print('likeli')
#         print(np.mean((np.log(D_but+1) - np.log(D_sim+1))**2))
    
    return (D_up, D_but)


def upper_lower_cases(packs, D_sim, ts, dt, n_train, CI = 0.95, debug=False):
    Ds = []
    likelis = []
    invalid = 0
    for alpha,beta,p in packs:

        if(alpha <0 or alpha >= 1 or beta <0 or p <0):
            invalid += 1
            print(invalid)
            continue
        if(debug): print(alpha, beta, p)
        D = ERF((alpha, beta, p, ts))


        likelis.append(np.mean((np.log(D_sim+1) - np.log(D+1))**2))

        Ds.append(D)
    Sample = len(Ds)
    print(Sample)
    order = sorted(range(len(likelis)), key=lambda k: likelis[k])
    
    Ds = [Ds[i] for i in order][0:round(Sample*CI)]
    Ds_mat = np.array(Ds)
    D_up =  np.max(Ds_mat[:, 1:]-Ds_mat[:, :-1],axis=0)
#     D_up[n_train-1] = max(np.max(Ds_mat[:, n_train]) - dt, np.max(Ds_mat[:, n_train-1])
    D_but =  np.min(Ds_mat[:, 1:]-Ds_mat[:, :-1],axis=0)
#     D_but[n_train] = np.min(Ds_mat[:, n_train-1]) - np.min(Ds_mat[:, n_train-2])
    

    return (D_up, D_but)


# In[4]:


def firstInfected(cum_Infect, N, wdate=False):
    i =  np.argmax(cum_Infect>N/np.exp(14))
    if(wdate): return i,self.dates[i]
    return i
    
    

    
def runERF(state, county, criter, loss, n_test = 0, pred_interval = 0, perday=False):
    try:
        N = Ns[(title.state == state)&(title.city == county)].to_numpy()[0]
    except:
        print(np.any(title.state == state)&(title.city == county))
        print(state)
        print(county)
    I = data[(title.state == state)&(title.city == county)].to_numpy()[0]
    D = death[(title.state == state)&(title.city == county)].to_numpy()[0]
    i_first = firstInfected(D, N)
    
    sDate = dates[i_first]
    eDate = dates[-1]
    train_dates = [d.date().strftime('%-m/%-d/%y') for d in pd.date_range(str(sDate),str(eDate),freq='D')]
    
#     D= D[i_first:]
    D = I[i_first:]
#     I = I[i_first:]
    
    flength = len(D)+pred_interval
    nt = len(D)
    print(nt)
    n_train = nt-n_test

    ts = np.linspace(0, nt-1, nt)
    D_train, D_test = D[:n_train], D[n_train:]
    
    t = np.linspace(0, flength-1, flength)
    n_train = len(D) - n_test
    
    D_train, D_test = D[:n_train], D[n_train:]
    
    p0 = 10
    a0 = 0.1
    b0 = 10
    _, alpha, beta, p, success = minimization((a0, b0, p0), (ts[0:n_train], D_train, N, loss))

    print('alpha: {}, beta: {}, p: {}'.format(alpha, beta, p))
    if success:
        print('success')
        
    
    if(pred_interval > 0):
        d = ERF((alpha, beta, p, t))
        return_dates = [d.date().strftime('%-m/%-d/%y') for d in pd.date_range(str(eDate), periods=pred_interval+1)][1:] 
        packs = bootstrap((D, t[:len(D)], alpha, beta, p, loss))
        if perday:
            upper, lower = upper_lower_cases(packs, d, t, D[-1], len(D))
            d = d[len(D):len(D)+pred_interval] - d[len(D)-1:len(D)+pred_interval-1]
            print('case per day')
            print(d)
            upper = upper[len(D)-1:len(D)+pred_interval-1]
            lower = lower[len(D)-1:len(D)+pred_interval-1]
        else:
            upper, lower = upper_lower(packs, d, t)
            d = d[len(D):len(D)+pred_interval]
            upper = upper[len(D):len(D)+pred_interval]
            lower = lower[len(D):len(D)+pred_interval]
        
    else:
        d = ERF((alpha, beta, p, t))
        return_dates = train_dates[n_train:]
        packs = bootstrap((D, t[:len(D)], alpha, beta, p, loss))
        if perday:
            upper, lower = upper_lower_cases(packs, d, t, D[n_train-1], n_train)
            d = d[len(D)-n_test:len(D)] - d[len(D)-n_test-1:len(D)-1]
            upper = upper[len(D)-n_test-1:len(D)-1]
            lower = lower[len(D)-n_test-1:len(D)-1]
            print(D[len(D)-n_test:len(D)])
            print(D[len(D)-n_test-1:len(D)-1])
            print('case per day:')
            print(d)
        else:
            upper, lower = upper_lower(packs, d, t)
            d = d[len(D)-n_test:len(D)]
            upper = upper[len(D)-n_test:len(D)]
            lower = lower[len(D)-n_test:len(D)]
    print('lower')
    print(lower)
    print('upper')
    print(upper)
    
    print(sDate, eDate)
    return (upper, lower, d, return_dates)


# In[6]:


# state= 'California'
# county = 'Los Angeles'

Lfuncs = ['weight']


criter = 'cases'
# measure = 'test'
country = 'US'

death_path = Path('./../STOPCOVID19TOGETHER/data/latest/us_county_deaths.csv/')
death_df = pd.read_csv(death_path)
death_cases = death_df.iloc[:,-1]
indexes = death_cases.argsort()[::-1]

if criter == 'cases':
    US_county_path = Path('./../STOPCOVID19TOGETHER/data/latest/us_county_cases.csv/')
else:
    US_county_path = Path('./../STOPCOVID19TOGETHER/data/latest/us_county_deaths.csv/')
    

    
df = pd.read_csv(US_county_path)
data = df.iloc[:,0].str.split(",").to_list()
death = death_df.iloc[:,2:]
new_df = pd.DataFrame(data)
new_df.columns = [ "city", "state", "country"]
new_df.loc[pd.isnull(new_df.country), "country"] = new_df[pd.isnull(new_df.country)].state
new_df = new_df.apply(lambda x: x.str.strip())
title = new_df
title = title.iloc[indexes,:].iloc[0:30,:]
data = df.iloc[:,2:]
data = data.iloc[indexes,:].iloc[0:30,:]
death = death.iloc[indexes,:].iloc[0:30,:]

Ns = df.iloc[:, 1]
Ns = Ns[indexes][0:30]

dates = np.array(df.columns.to_list()[2:])

i_err = 0
pred_interval = 30
test_interval = 7
perday = True

for loss in Lfuncs:
#     for measure in ['predictions', 'test']:
    for measure in ['test','predictions']:
        if measure == 'test':
            interval = test_interval
        else:
            interval = pred_interval
        df_output = pd.DataFrame(columns=list(range(interval+1)))
        df_output_u = pd.DataFrame(columns=list(range(interval+1)))
        df_output_l = pd.DataFrame(columns=list(range(interval+1)))
        for state in np.unique(title.state):
#         for state in ['California']:
            for county in np.unique(title[title.state == state].city):
                if measure == 'test':
                    upper, lower, d, return_dates = runERF(state, county, criter, loss, n_test = test_interval,perday=perday)
                else:
                    upper, lower, d, return_dates = runERF(state, county, criter, loss, pred_interval = pred_interval,perday=perday)
                df_output.loc[i_err] = [f'{county}, {state}, {country}'] + list(d[-interval:])
                df_output_u.loc[i_err] = [f'{county}, {state}, {country}'] + list(upper[-interval:])
                df_output_l.loc[i_err] = [f'{county}, {state}, {country}'] + list(lower[-interval:])

                i_err += 1

        if measure == 'predictions':
            x_pred_dates = pd.date_range(return_dates[0],periods=len(return_dates),freq='D')[-pred_interval:]
        else:
            x_pred_dates = pd.date_range(return_dates[0],periods=len(return_dates),freq='D')[-test_interval:]
            
        for df_out,name in zip([df_output,df_output_u,df_output_l],['prediction','prediction_upper_bound','prediction_lower_bound']):  
            df_out.columns = [''] + list(x_pred_dates.strftime('%-m/%-d/%y'))
            if perday:
                df_out.to_csv(f'./../STOPCOVID19TOGETHER/data/predictions/evals/evals_{measure}/UW_{loss}/us_county_cases_{name}_perday.csv',index=False)
            else:
                df_out.to_csv(f'./../STOPCOVID19TOGETHER/data/predictions/evals/evals_{measure}/UW_{loss}/us_county_cases_{name}.csv',index=False)
        else:
            
            for df_out,name in zip([df_output,df_output_u,df_output_l],['prediction','prediction_upper_bound','prediction_lower_bound']):  
                df_out.columns = [''] + list(x_pred_dates.strftime('%-m/%-d/%y'))
                if perday:
                    df_out.to_csv(f'./../STOPCOVID19TOGETHER/data/predictions/evals/evals_{measure}/UW_{loss}/us_county_cases_{name}_perday.csv',index=False)
                else:
                    df_out.to_csv(f'./../STOPCOVID19TOGETHER/data/predictions/evals/evals_{measure}/UW_{loss}/us_county_cases_{name}.csv',index=False)
                    
                    
                    
                    


# In[6]:


fig,ax = plt.subplots(figsize=(12,8))
ax.fill_between(np.arange(0, len(upper)),y1=upper,y2=lower,alpha=0.2)
ax.plot(d,'--r', label='True')
# ax.plot(d,label='WU')

ax.legend()
ax.set_title('Narrow Prior')

ax.set_xlim((0,len(d)))
ax.set_ylim((0, np.max(upper)))

plt.show()


# In[33]:



x = np.sort(np.random.choice(np.arange(20-7,20-1), 5,replace=False))


# In[35]:


x[2:-1]


# In[36]:


x


# In[ ]:






31
alpha: 0.08886204417445699, beta: 12.272249530160126, p: 11614.4499175303
success
0.08886204417445699 12.272249530160126 11614.4499175303
200
[11400 12021 12341 13823 15153 16447 17537]
[10854 11400 12021 12341 13823 15153 16447]
case per day:
[215.33058259 178.96718018 146.41693116 117.91238566  93.4711899
  72.93673726  56.02283781]
lower
[0. 0. 0. 0. 0. 0. 0.]
upper
[1448.08686184 1492.88148605 1493.94510152 1451.18417124 1368.32421165
 1252.44637847 1277.17987362]
3/24/20 4/23/20
37
alpha: 0.08334561804747605, beta: 19.667568455494816, p: 7770.442786710034
success
0.08334561804747605 19.667568455494816 7770.442786710034


KeyboardInterrupt: 